In [1244]:
playersPath = "I:/FM/Football Manager 2023/graphics/1999/"
shortlist = "I:/FM/Football Manager 2023/Shortlist.html"
rename = False

In [1245]:
import os, os.path, pathlib

In [1246]:
from lxml import etree as ET

In [1247]:
import pandas as pd

In [1248]:
newgens = pd.read_html(shortlist,encoding="UTF-8")[0]

In [1249]:
newgens = newgens.drop(["Rec","Inf"], axis=1)


In [1250]:
print(newgens)

             UID               Name                  Club
0     2002116244  Tommaso Abbrescia                  Bari
1     2002144788      Miguel Aceval             Colo Colo
2     2002144802     Albert Acevedo  Universidad Católica
3     2002152485  Emmanuel Adebayor               FC Metz
4     2002107683         Adem Koçak            Ankaragücü
...          ...                ...                   ...
1110  2002115579      Daniel Ziebig        Dynamo Dresden
1111  2002110370      François Zoko                 Nancy
1112  2002152859        Ramon Zomer             FC Twente
1113  2002144492               Zulu         Novo Hamburgo
1114  2002112979     Theo Zwarthoed                    AZ

[1115 rows x 3 columns]


In [1251]:
newgens["Deaccented"] = newgens["Name"].str.normalize("NFKD").str.encode("ascii",errors="ignore").str.decode("utf-8")

In [1252]:
duplicates = newgens[newgens["Name"].duplicated(keep=False)]
noDuplicates = newgens[~newgens["Name"].duplicated(keep=False)]

In [1253]:

print(noDuplicates)

             UID               Name                  Club         Deaccented
0     2002116244  Tommaso Abbrescia                  Bari  Tommaso Abbrescia
1     2002144788      Miguel Aceval             Colo Colo      Miguel Aceval
2     2002144802     Albert Acevedo  Universidad Católica     Albert Acevedo
3     2002152485  Emmanuel Adebayor               FC Metz  Emmanuel Adebayor
4     2002107683         Adem Koçak            Ankaragücü         Adem Kocak
...          ...                ...                   ...                ...
1110  2002115579      Daniel Ziebig        Dynamo Dresden      Daniel Ziebig
1111  2002110370      François Zoko                 Nancy      Francois Zoko
1112  2002152859        Ramon Zomer             FC Twente        Ramon Zomer
1113  2002144492               Zulu         Novo Hamburgo               Zulu
1114  2002112979     Theo Zwarthoed                    AZ     Theo Zwarthoed

[1101 rows x 4 columns]


In [1254]:
files = list(pathlib.Path(playersPath).glob("*.[pj][np][g]"))

In [1255]:

fileData = pd.DataFrame({"FilePath":files})

In [1256]:
fileData["FileName"] = fileData["FilePath"].apply(lambda file: os.path.splitext(os.path.basename(file))[0] )
print(fileData)

                                              FilePath            FileName
0    I:\FM\Football Manager 2023\graphics\1999\2002...          2002113575
1    I:\FM\Football Manager 2023\graphics\1999\Abdu...   Abdullah Fusseini
2    I:\FM\Football Manager 2023\graphics\1999\Adam...          Adam Virgo
3    I:\FM\Football Manager 2023\graphics\1999\Adem...          Adem Kocak
4    I:\FM\Football Manager 2023\graphics\1999\Adna...       Adnan Barakat
..                                                 ...                 ...
427  I:\FM\Football Manager 2023\graphics\1999\Youn...  Youness Bengelloun
428  I:\FM\Football Manager 2023\graphics\1999\Yuri...                Yuri
429  I:\FM\Football Manager 2023\graphics\1999\Zbyn...      Zbynek Pospech
430  I:\FM\Football Manager 2023\graphics\1999\Ze R...              Ze Rui
431  I:\FM\Football Manager 2023\graphics\1999\Zesh...         Zesh Rehman

[432 rows x 2 columns]


In [1257]:
if(fileData["FileName"].str.contains("_").any()):
    fileData[["Identifier", "Team"]] = fileData["FileName"].str.split(pat="_",expand=True)
else:
    fileData["Identifier"] = fileData["FileName"]
    fileData["Team"] = None
print(fileData)

                                              FilePath            FileName  \
0    I:\FM\Football Manager 2023\graphics\1999\2002...          2002113575   
1    I:\FM\Football Manager 2023\graphics\1999\Abdu...   Abdullah Fusseini   
2    I:\FM\Football Manager 2023\graphics\1999\Adam...          Adam Virgo   
3    I:\FM\Football Manager 2023\graphics\1999\Adem...          Adem Kocak   
4    I:\FM\Football Manager 2023\graphics\1999\Adna...       Adnan Barakat   
..                                                 ...                 ...   
427  I:\FM\Football Manager 2023\graphics\1999\Youn...  Youness Bengelloun   
428  I:\FM\Football Manager 2023\graphics\1999\Yuri...                Yuri   
429  I:\FM\Football Manager 2023\graphics\1999\Zbyn...      Zbynek Pospech   
430  I:\FM\Football Manager 2023\graphics\1999\Ze R...              Ze Rui   
431  I:\FM\Football Manager 2023\graphics\1999\Zesh...         Zesh Rehman   

             Identifier  Team  
0            2002113575  None  

In [1258]:
config = ET.Element("record")
preload = ET.SubElement(config,"boolean", id="preload", value="false")
amap = ET.SubElement(config,"boolean", id="amap", value="false")
playerList = ET.SubElement(config, "list", id="maps")


In [1259]:
for index,fileRow in fileData.iterrows():
    record = {"from":None, "to":None}
    identifier = str(fileRow["Identifier"])
    fileName = fileRow["FileName"]
    if(fileRow["Team"]):
        player = newgens.loc[((newgens["Name"] == identifier)|(newgens["Deaccented"] == identifier)) & (newgens["Club"] == fileRow["Team"])]
        
    else:
        
        player = noDuplicates.loc[(noDuplicates["Name"] == identifier)|(noDuplicates["Deaccented"] == identifier)|(noDuplicates["UID"].astype(str) == identifier)]
        
    if(len(player) == 1 ):
        fileRow["Found"] = True
        print(player)
        if(rename and fileName != str(player["UID"].values[0]) ):
            path, name = os.path.split(fileRow["FilePath"])
            extension = os.path.splitext(name)[1]
            newFile = str(player["UID"].values[0]) + extension
            newPath = os.path.join(path,newFile)
            fileName = str(player["UID"].values[0])
            os.rename(fileRow["FilePath"], newPath)
        
        record = {"from":fileName,"to":str(player["UID"].values[0])}
    else:
        fileRow["Found"] = False
        
    if record["from"] and record["to"]:
        recordXML = ET.SubElement(playerList, "record")
        recordXML.set("from", str(record["from"]))
        recordXML.set("to",f'graphics/pictures/person/{record["to"]}/portrait')
    
        

            UID               Name     Club         Deaccented
480  2002113575  Abderrazzak Jadid  Brescia  Abderrazzak Jadid
            UID               Name    Club         Deaccented
365  2002113474  Abdullah Fusseini  Torino  Abdullah Fusseini
             UID        Name      Club  Deaccented
1052  2002116555  Adam Virgo  Brighton  Adam Virgo
          UID        Name        Club  Deaccented
4  2002107683  Adem Koçak  Ankaragücü  Adem Kocak
          UID           Name       Club     Deaccented
6  2002113015  Adnan Barakat  NAC Breda  Adnan Barakat
            UID              Name                 Club        Deaccented
392  2002116878  Agustín González  Rychnov nad Kneznou  Agustin Gonzalez
            UID         Name     Club   Deaccented
269  2002111511  Aïmen Demai  FC Metz  Aimen Demai
            UID           Name    Club     Deaccented
762  2002113472  Akeem Omolade  Torino  Akeem Omolade
          UID            Name     Club      Deaccented
5  2002117400  Alade Adeshi

In [1260]:

configTree = config.getroottree()
configTree.write(f"{playersPath}/config.xml", encoding="utf-8", pretty_print=True)